# wordslab-notebooks-lib.tools

> Predefined tools for LLMs

In [4]:
#| default_exp tools

## Mistral vibe bultin tools

The following tools code is inspired from **Mistral Vibe 2.0** (Apache 2.0 licence) :

https://github.com/mistralai/mistral-vibe/tree/main/vibe/core/tools/builtins